## Imports and constants

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
PROPENSITIES = {'original': "Original", 'bb-item-user': "BB Item-User", 'bb-item': 'BB Item'}
RESULTS_FILE = 'aoa_all.csv'

In [3]:
METHODS = { 'bpr': 'BPR',
            'ubpr' : 'UBPR',
            'dubpr': 'DUBPR',
            'relmf' : 'RelMF',
            'dumf': 'DUMF',
            'expomf' : 'ExpoMF',
            'ip' : 'ItemPop',
            'wmf' : 'WMF',
        }

## Define functions for loading and plotting results

In [4]:
def load_df(results_path, data, propensity, method, results_file):
    try :
        df = pd.read_csv(f"{results_path}/{data}/{propensity}/{method}/results/{results_file}")
    except:
        print(f"Error loading {results_path}/{data}/{propensity}/{method}/results/{results_file}")
        print(f"It is required to run the models for {data} dataset with {propensity} propensity and {method} method")
        raise Exception("Missing results")
    df = df.rename(columns={'Unnamed: 0': 'Metric@K'})
    metrics =  df['Metric@K'].to_list()
    df = df.iloc[:, 1:].transpose()
    df.columns = metrics
    df.reset_index(inplace=True, drop=True)
    df['Method'] = method
    df = df.melt(id_vars=['Method'], var_name='Metric@K', value_name='Value')
    df['Propensity'] = PROPENSITIES[propensity]
    df['Metric'] = df['Metric@K'].apply(lambda x: x.split('@')[0])
    df['K'] = df['Metric@K'].apply(lambda x: x.split('@')[1])
    return df

In [5]:
def load_results(results_path, data):
    """Load all results for a given dataset"""
    dfs = []
    for propensity in PROPENSITIES.keys():
        for method in METHODS.keys():
            df = load_df(results_path, data, propensity, method, RESULTS_FILE)
            dfs.append(df)
    return pd.concat(dfs)

def plot_metric(df, metric, methods=None, propensities=None):
    """Plot a metric for given methods and propensities"""
    if methods:
        df = df[df['Method'].isin(methods)]
    if propensities:
        df = df[df['Propensity'].isin(propensities)]
    df = df[df['Metric'] == metric]
    fig = px.box(df, x="Metric@K", y="Value", color="Method", facet_col="Propensity")
    fig.update_traces(quartilemethod="exclusive")
    fig.for_each_xaxis(lambda x: x.update({'title': f"{metric}@K"}) if x.title.text == 'Metric@K' else x)
    fig.show()


In [6]:
_propensities = ['Original', 'BB Item-User']
_methods = ['bpr', 'dubpr', 'dumf',  'relmf', 'ubpr']

## Plot metrics for Coat dataset

In [7]:
coat_df =load_results("logs", "coat")

### DCG@K

In [8]:
plot_metric(coat_df, 'DCG', methods=_methods, propensities=_propensities)


### MAP@K

In [9]:
plot_metric(coat_df, 'MAP', methods=_methods, propensities=_propensities)


### Recall@K

In [10]:
plot_metric(coat_df, 'Recall', methods=_methods, propensities=_propensities)


## Plot metrics for Yahoo! C3 dataset

In [11]:
yahoo_df =load_results("logs", "yahoo")

### DCG@K

In [12]:
plot_metric(yahoo_df, 'DCG', methods=_methods, propensities=_propensities)


### MAP@K

In [13]:
plot_metric(yahoo_df, 'MAP', methods=_methods, propensities=_propensities)


### Recall@K

In [14]:
plot_metric(yahoo_df, 'Recall', methods=_methods, propensities=_propensities)
